### testing the base wav2vec model from the huggingface transformer api

In [1]:
# download & extract test dataset from librispeech
import urllib.request
import tarfile
import os
if not os.path.exists('./data/LibriSpeech'):
    print('downloading test dataset (300mb)')
    urllib.request.urlretrieve("https://www.openslr.org/resources/12/test-clean.tar.gz", "test.tar.gz")

    print('extracting data')
    file = tarfile.open('test.tar.gz')
    file.extractall('./data')
    file.close()

downloading test dataset (300mb)
extracting data


In [2]:
# source from https://colab.research.google.com/drive/1dnNrGy1U260L403OuhTsDjBQkdGHmvL9
#import jiwer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import nltk
import librosa
import torch
import soundfile as sf
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def load_wav2vec_960h_model():
    """
    Returns the tokenizer and the model from pretrained tokenizers models
    """
    tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")    
    return tokenizer, model

def correct_uppercase_sentence(input_text): 
    """
    Returns the corrected sentence
    """  
    sentences = nltk.sent_tokenize(input_text)
    return (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))

In [4]:
def asr_transcript(tokenizer, model, input_file):
    """
    Returns the transcript of the input audio recording

    Output: Transcribed text
    Input: Huggingface tokenizer, model and wav file
    """
    #read the file
    speech, samplerate = sf.read(input_file)
    #make it 1-D
    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]
    #Resample to 16khz
    if samplerate != 16000:
        speech = librosa.resample(speech, samplerate, 16000)
    #tokenize
    input_values = tokenizer(speech, return_tensors="pt").input_values
    #take logits
    logits = model(input_values).logits
    #take argmax (find most probable word id)
    predicted_ids = torch.argmax(logits, dim=-1)
    #get the words from the predicted word ids
    transcription = tokenizer.decode(predicted_ids[0])
    #output is all uppercase, make only the first letter in first word capitalized
    transcription = correct_uppercase_sentence(transcription.lower())
    return transcription

In [5]:
#loading the models
tokenizer, model = load_wav2vec_960h_model()

#transcript
f = open("./data/LibriSpeech/test-clean/61/70968/61-70968.trans.txt", "r")
lines= str.splitlines(f.read())

#goes through the audio files in the chapter
for i in range(0,63):
    filename='./data/LibriSpeech/test-clean/61/70968/61-70968-00'+str(i).zfill(2)
    print(lines[i].split(' ', 1)[1])
    txt=lines[i].split(' ', 1)[1]
    wav_input = filename+'.flac'
    
    text = asr_transcript(tokenizer,model,wav_input)
    print(text)
    print('-----------------------------------')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
C:\Users\nguye\anaconda3\envs\envDL\lib\site-packages\transformers\models\wav2vec2\tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HE BEGAN A CONFUSED COMPLAINT AGAINST THE WIZARD WHO HAD VANISHED BEHIND THE CURTAIN ON THE LEFT
He began a confused complaint against the wizard who had vanished behind the curtain on the left
-----------------------------------
GIVE NOT SO EARNEST A MIND TO THESE MUMMERIES CHILD
Give not so earnest a mind to these mummories child
-----------------------------------
A GOLDEN FORTUNE AND A HAPPY LIFE
A golden fortune and a happy life
-----------------------------------
HE WAS LIKE UNTO MY FATHER IN A WAY AND YET WAS NOT MY FATHER
He was like unto my father in a way and yet was not my father
-----------------------------------
ALSO THERE WAS A STRIPLING PAGE WHO TURNED INTO A MAID
Also there was a strippling page who turned into a maid
-----------------------------------
THIS WAS SO SWEET A LADY SIR AND IN SOME MANNER I DO THINK SHE DIED
This was so sweet a lady sir and in some manner i do think she died
-----------------------------------
BUT THEN THE PICTURE WAS GONE AS QUICKLY AS IT 

Pray follow us with mine and my lord sheriff's men
-----------------------------------
NOTTINGHAM CASTLE WAS REACHED AND ADMITTANCE WAS DEMANDED
Nottingham castle was reached an admittance was demanded
-----------------------------------
MASTER MONCEUX THE SHERIFF OF NOTTINGHAM WAS MIGHTILY PUT ABOUT WHEN TOLD OF THE RIOTING
Master monceux the sheriff of nottingham was mightily put about when told of the rioting
-----------------------------------
AND HENRY MIGHT RETURN TO ENGLAND AT ANY MOMENT
And henry might return to england at any moment
-----------------------------------
HAVE YOUR WILL CHILD IF THE BOY ALSO WILLS IT MONTFICHET ANSWERED FEELING TOO ILL TO OPPOSE ANYTHING VERY STRONGLY JUST THEN
Have your will child if the boy also wills it montfighet answered feeling too ill to oppose anything very strongly just then
-----------------------------------
HE MADE AN EFFORT TO HIDE HIS CONDITION FROM THEM ALL AND ROBIN FELT HIS FINGERS TIGHTEN UPON HIS ARM
He made an effort to hide hi